In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from functools import wraps
from datetime import timedelta
import os
import json


In [ ]:
# JSON 파일을 열고 데이터 읽기
with open('./static\data/calendar.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

temp2=[]
for i in range(len(data)):
    temp = data[i]

    if temp['start']>='2024-12-01':
        temp2 = np.append(temp2,[temp])
for i in range(len(temp2)):
    temp2[i]['description2']= ""
    for tmp in temp2[i]['description'].split("\n작업장"):

        if "undefined" not in tmp:
            if temp2[i]['description2']!="":
                temp2[i]['description2'] = temp2[i]['description2']+"\n"+tmp
            else: 
                temp2[i]['description2'] = tmp
temp2[2]['description2'] = temp2[2]['description2'].replace("\n연장근무","")
temp2[2]['description2'] = temp2[2]['description2'][:-3]
temp2[3]['description2'] = temp2[3]['description2'].replace("\n연장근무","")
temp2[3]['description2'] = temp2[3]['description2'][:-3]

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Border, Side,Font

In [195]:
iter_sheet = [50 * i for i in range(0,30)]
wb = Workbook()
ws = wb.active
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)


ws.title = "작업일지"


for iter in range(len(temp2)):
    print(iter)
    today_worker = []
    cell_row = iter_sheet[iter]
    ws.merge_cells("A"+str(cell_row+1)+":S"+str(cell_row+1))  # A1 ~ F1까지 병합
    ws["A"+str(cell_row+1)] = "작 업 일 지"
    ws["A"+str(cell_row+1)].alignment = Alignment(horizontal='center')
    ws["A"+str(cell_row+1)].font = Font(size=14, bold=True)

    temp_parse = temp2[iter]['description2'].split("작업장")[1:]
    print(temp_parse)
    parse_i = temp_parse[0].split("\n")
    site= parse_i[0].split(":")[1].replace(" ","") ##작업장
    start = temp2[iter]['start']

    ws.merge_cells("A"+str(cell_row+2)+":S"+str(cell_row+2))  # A1 ~ F1까지 병합
    ws["A"+str(cell_row+2)] = "현장명 : " + site
    ws["A"+str(cell_row+2)].alignment = Alignment(horizontal='center')
    ws.merge_cells("A"+str(cell_row+3)+":D"+str(cell_row+3))  # A1 ~ F1까지 병합
    ws.merge_cells("E"+str(cell_row+3)+":S"+str(cell_row+3))  # A1 ~ F1까지 병합
    ws["A"+str(cell_row+3)] = "날짜 : " + start.split(" ")[0]

    ws.append(['NO', '공종', '성명', '공수', '# 주간작업', '', '', '', '', '', '', '', '', '', '', '', '', '', ''])
    ws.merge_cells("E"+str(cell_row+4)+":S"+str(cell_row+4))  # 국적 병합
    ws["E"+str(cell_row+4)].alignment = Alignment(horizontal='center')

    ws.append(['', '', '', '', '구간', '층', '형태', '작업 단가별 분류', '작업내용', '공종', '공수', '작업명단', '', '', '', '', '', '', ''])
    ws["L"+str(cell_row+4)].alignment = Alignment(horizontal='center')
    ws.merge_cells("S"+str(cell_row+5)+":S"+str(cell_row+5)) 
    full_number =0 

    for jj in range(cell_row+6,+cell_row+24):
        ws.merge_cells("L"+str(jj)+":S"+str(jj))

    for i in range(len(temp_parse)):
        parse_i = temp_parse[i].split("\n")
        print(parse_i)
        ##E~K 6부터 
        number = int(parse_i[1].split(":")[1].replace(" ","")) ## 공수 
        full_number = full_number+number ## 합계 

        work_parse = parse_i[2].split(":")[1].replace(" ","") ## 
        work1 = work_parse.split("_")[1] ##형태 
        floor = work_parse.split("_")[2] ## 층 
        floor2 = work_parse.split("_")[3] ## 구간
        work2 = work_parse.split("_")[4] ## 단가별 분류
        work3 = work_parse.split("_")[5] ## 작업내용 
        # for aa in ['E','F','G','H','I','J','K']:
        ws['E'+str(6+i+cell_row)] = floor2
        ws['F'+str(6+i+cell_row)] = floor
        ws['G'+str(6+i+cell_row)] = work1
        ws['H'+str(6+i+cell_row)] = work2
        ws['I'+str(6+i+cell_row)] = work3
        ws['J'+str(6+i+cell_row)] = '형틀'
        ws['K'+str(6+i+cell_row)] = number

        ## A~D 5부터 
        for j in range(5,len(parse_i)):
            print(j)
            today_worker = today_worker+[{parse_i[j].split(":")[1].split("|")[0].replace(" ",""):parse_i[j].split(":")[1].split("|")[1].replace(" ","")}]
            print(today_worker)

    ws.merge_cells("E"+str(cell_row+28)+":J"+str(cell_row+28))
    ws["E"+str(cell_row+28)]='합계'
    ws["K"+str(cell_row+28)] = full_number
    ws.merge_cells("L"+str(cell_row+28)+":S"+str(cell_row+28))
    ws.merge_cells("E"+str(cell_row+28)+":S"+str(cell_row+28))
    ws["E"+str(cell_row+29)] = "# 연장작업"
    ws["E"+str(cell_row+30)] = '구간'
    ws["F"+str(cell_row+30)] = '층'
    ws["G"+str(cell_row+30)] = '형태'
    ws["H"+str(cell_row+30)] = '작업 단가별 분류'
    ws["I"+str(cell_row+30)] = '작업내용'
    ws["J"+str(cell_row+30)] = '공종'
    ws["K"+str(cell_row+30)] = '공수'
    ws.merge_cells("L"+str(cell_row+30)+":S"+str(cell_row+30))
    ws["L"+str(cell_row+30)] = '작업 명단'


    for idx,k in enumerate(today_worker):
        ws['A'+str(5+idx+cell_row)] = idx+1
        ws['B'+str(5+idx+cell_row)] = '형틀'
        ws['C'+str(5+idx+cell_row)] = list(k.keys())[0]
        ws['D'+str(5+idx+cell_row)] = list(k.values())[0]
        last_k = idx
        last_idx = (5+idx+cell_row)

    for i,idx in enumerate(range(last_idx,44+cell_row)):
        ws['A'+str(idx)] = last_k+i
        ws['B'+str(idx)] = '형틀'

    ws["A"+str(cell_row+44)] = '합계'
    ws.merge_cells("A"+str(cell_row+44)+":C"+str(cell_row+44))
    ws["D"+str(cell_row+44)] = full_number
    ws.merge_cells("E"+str(cell_row+44)+":J"+str(cell_row+44))
    ws["E"+str(cell_row+44)] = '합계'
    ws.merge_cells("L"+str(cell_row+44)+":S"+str(cell_row+44))
    ws["K"+str(cell_row+44)] = full_number
    ws["A"+str(cell_row+45)] = "# 특이사항"
    ws.merge_cells("A"+str(cell_row+45)+":S"+str(cell_row+45))
    for kk in range(46+cell_row,50+cell_row):
        ws.merge_cells('A'+str(kk)+':S'+str(kk))
        
    alignment_center = Alignment(horizontal='center', vertical='center', wrap_text=False)
    # (2) 모든 사용된 셀 범위를 조회 (1~ws.max_row행, 1~8열) 후 테두리/정렬 적용
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=19):
        for cell in row:
            # 테두리
            cell.border = thin_border
            cell.alignment = alignment_center




0
[' : 김포 \n 작업인원 : 1 \n작업내용 : 김포_직영_B4_D_코어_자재양중 및 야기리 설치 작업 신호수_1_100%\n특이사항 : \n테이블 데이터:\n행 1: 모용국 | 1 | 100.00% | 김포_직영_B4_D_코어_자재양중 및 야기리 설치 작업 신호수_1_100% | 삭제 |']
[' : 김포 ', ' 작업인원 : 1 ', '작업내용 : 김포_직영_B4_D_코어_자재양중 및 야기리 설치 작업 신호수_1_100%', '특이사항 : ', '테이블 데이터:', '행 1: 모용국 | 1 | 100.00% | 김포_직영_B4_D_코어_자재양중 및 야기리 설치 작업 신호수_1_100% | 삭제 |']
5
[{'모용국': '1'}]
1
[' : 김포 \n 작업인원 : 2 \n작업내용 : 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100%\n특이사항 : \n테이블 데이터:\n행 1: 김승학 | 1 | 50.00% | 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100% | 삭제 |\n행 2: 오홍성 | 1 | 50.00% | 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100% | 삭제 |']
[' : 김포 ', ' 작업인원 : 2 ', '작업내용 : 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100%', '특이사항 : ', '테이블 데이터:', '행 1: 김승학 | 1 | 50.00% | 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100% | 삭제 |', '행 2: 오홍성 | 1 | 50.00% | 김포_도급_B1_A_램프_1번, 보 제작, 슬라브 작업_2_100% | 삭제 |']
5
[{'김승학': '1'}]
6
[{'김승학': '1'}, {'오홍성': '1'}]
2
[' : 김포 \n 작업인원 : 5 \n작업내용 : 김포_도급_B4_D_램프_슬라브 설치_5_100%\n특이사항 : 특이사항: 명일 타설: \n테이블 데이터:\n행 1: 이영남 | 1 | 20.00% | 김포_

In [196]:
wb.save("작업일지.xlsx")

In [93]:
last_idx

15

In [ ]:
for idx in range(last_idx,39):
    ws['A'+str(5+idx)] = idx-5+1
    ws['B'+str(5+idx)] = '형틀'

15

In [23]:
parse_log_data(temp2[4]['description'].split("\n"))

[{'work_place': '',
  'worker_count': 'undefined',
  'work_content': '',
  'rows': []}]

In [ ]:
temp2_split = temp2[4]['description'].split("\n")
## 작업장 
site = temp2_split[0].split(":")[1].replace(" ","")
date = temp2[3]['start']



['작업장 : 김포 ',
 ' 작업인원 : 4 ',
 '작업내용 : 김포_도급_1_A_코어_본관동 옥탑 사무동 패드 및 난간 작업_4_100%',
 '특이사항 : ',
 '테이블 데이터:',
 '행 1: 신영조 | 1 | 25.00% | 김포_도급_1_A_코어_본관동 옥탑 사무동 패드 및 난간 작업_4_100% | 삭제 |',
 '행 2: 조병칠 | 1 | 25.00% | 김포_도급_1_A_코어_본관동 옥탑 사무동 패드 및 난간 작업_4_100% | 삭제 |',
 '행 3: 신성일 | 1 | 25.00% | 김포_도급_1_A_코어_본관동 옥탑 사무동 패드 및 난간 작업_4_100% | 삭제 |',
 '행 4: 신영호 | 1 | 25.00% | 김포_도급_1_A_코어_본관동 옥탑 사무동 패드 및 난간 작업_4_100% | 삭제 |',
 '']

In [ ]:
insadb = pd.read_excel("./static/data/INSA_DB.xlsx")

insadb_json = insadb.to_json(orient='records',)

In [7]:
## 작업 key만들기 

In [39]:
import random
import string
def new_generate_random_key(df):
    """알파벳 3글자 + 숫자 4자리 난수 생성 (예: ABC1234)"""
    
    letters = ''.join(random.choices(string.ascii_uppercase, k=3))
    digits = ''.join(random.choices(string.digits, k=4))
    newkey = letters + digits

    if newkey in df['KEY'].unique():
        while newkey in df['KEY'].unique():
            print("겹치는거 있음?")
            letters = ''.join(random.choices(string.ascii_uppercase, k=3))
            digits = ''.join(random.choices(string.digits, k=4))
            newkey = letters + digits
        return newkey
    else:
        return newkey
    

In [ ]:
work_info.to_excel("./static/data/작업공수/작업입력.xlsx", index=False)

In [55]:
import pandas as pd 
work_info = pd.read_excel("./static/data/작업공수/작업계획.xlsx")

In [58]:
dup = work_info.loc[(work_info[key_column]==new_val[key_column]).sum(axis=1)==6]
new_val['KEY'] =  new_generate_random_key(work_info)
if len(dup)!=0:
    print('error')
else:
    work_info = pd.concat([work_info,new_val]).reset_index(drop=True)
work_info.loc[(work_info[key_column]==key_unique.values[i]).sum(axis=1)==6,'KEY'] = new_generate_random_key(work_info)

In [64]:
work_info.loc[(work_info[key_column]==new_val[key_column].values).sum(axis=1)==6]

,작업날짜,작업장,위치_분류1,위치_분류2,작업_분류1,작업_분류2,작업_분류3,연장,수량,단위,목표공수,도급/직영,KEY
12,2025-01-03,마곡,11,C,??,NaN,코어 벽체 부분 해체 기준목 설치 작업,NaN,NaN,NaN,30,직영,KNW7505


In [61]:
new_val = work_info.iloc[-1:,:]

In [62]:
new_val

,작업날짜,작업장,위치_분류1,위치_분류2,작업_분류1,작업_분류2,작업_분류3,연장,수량,단위,목표공수,도급/직영,KEY
12,2025-01-03,마곡,11,C,??,NaN,코어 벽체 부분 해체 기준목 설치 작업,NaN,NaN,NaN,30,직영,KNW7505


In [ ]:
key_column = ['작업장', '위치_분류1', '위치_분류2', '작업_분류1', '도급/직영', '작업_분류3']
work_info.loc[(work_info[key_column]==key_unique.values[i]).sum(axis=1)==6,'KEY'] =  new_generate_random_key(work_info)

In [2]:
import pandas as pd 
work_info = pd.read_excel("./static/data/작업공수/작업입력.xlsx")
# work_info['KEY']=""

In [5]:
work_info['작업날짜'].month

AttributeError: 'Series' object has no attribute 'month'

In [10]:
import datetime

In [1]:
import pandas as pd
insadb = pd.read_excel("./static/data/INSA_DB.xlsx")
insadb_json = insadb[['성명','작업장']].to_json(orient='records', force_ascii=False)

In [ ]:
cols = [x for x in insadb.columns if '2025-01-31' in x]
insadb[~insadb[cols[0]].isna()]

In [19]:
insadb[cols[0]].isna()

0        True
1        True
2       False
3        True
4        True
        ...  
1080     True
1081     True
1082     True
1083     True
1084     True
Name: 2025-01-31(출근), Length: 1085, dtype: bool

In [2]:
import pandas as pd

In [11]:
test = pd.read_csv("./static/data/test.csv").to_json(orient='records', force_ascii=False)

In [13]:
import json
file_path = "static/data/test.json"
with open(file_path, 'w',encoding='utf-8') as outfile:
    json.dump(test, outfile, ensure_ascii=False, indent=4)

In [9]:
setting = pd.read_excel("./static/data/작업공수/SETTING.xlsx").to_json(orient='records', force_ascii=False)

In [10]:
import json
file_path = "static/data/작업공수/SETTING.json"
with open(file_path, 'w',encoding='utf-8') as outfile:
    json.dump(setting, outfile, ensure_ascii=False, indent=4)

In [27]:
import numpy as np

In [29]:
today = datetime.datetime.today().strftime("%Y-%m")
work_info[work_info['작업날짜'].str.contains(today).replace(np.NaN,False)]

C:\Users\windows11\AppData\Local\Temp\ipykernel_5040\1848569744.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  work_info[work_info['작업날짜'].str.contains(today).replace(np.NaN,False)]


,작업날짜,작업장,위치_분류1,위치_분류2,작업_분류1,작업_분류2,작업_분류3,연장,수량,단위,...,목표공수,공수,인원,도급/직영,특이사항,작업,기성날짜,기성 비고,작업진행률,KEY
807,2025-01-03,마곡,5,C,??,NaN,바닥 및 보 거푸집 설치,NaN,NaN,NaN,...,6.0,4.0,3.0,도급,NaN,NaN,NaT,NaN,65%,XSZ1339
808,2025-01-03,마곡,12,C,??,NaN,코어 벽체 알폼 자재 양중 작업,NaN,NaN,NaN,...,4.0,4.0,3.0,도급,NaN,NaN,NaT,NaN,100%,VGB9612
809,2025-01-03,마곡,12,C,??,NaN,코어 벽체 알폼 자재 양중 작업,NaN,NaN,NaN,...,4.0,4.0,3.0,도급,NaN,NaN,NaT,NaN,0.75,VGB9612
810,2025-01-12,마곡,8,B,??,NaN,바닥 및 보 서포트 해체 작업,NaN,NaN,NaN,...,3.0,3.0,3.0,직영,NaN,NaN,NaT,NaN,0.65,NBZ5197
811,2025-01-12,마곡,11,B,??,NaN,코어 벽체 하부자키 및 요꼬 다대 작업,NaN,NaN,NaN,...,1.0,1.0,3.0,도급,NaN,NaN,NaT,NaN,100%,XVB9362
812,2025-01-03,마곡,11,C,??,NaN,코어 벽체 부분 해체 기준목 설치 작업,NaN,NaN,NaN,...,30.0,30.0,NaN,직영,NaN,NaN,NaT,NaN,0.0%,KNW7505
813,2025-01-27,마곡,11,B,NaN,바닥 및 보,단열재 자재 준비 작업,NaN,NaN,NaN,...,NaN,3.0,NaN,도급,NaN,NaN,NaT,NaN,0%,XGP1279
814,2025-01-27,마곡,11,A,NaN,바닥 및 보,용접 작업,NaN,NaN,NaN,...,NaN,3.0,NaN,도급,NaN,NaN,NaT,NaN,0%,LXJ0778
